In [ ]:
# 合成数集的分类
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.tree import DecisionTreeClassifier
#make_classification生成随机的二分类数据
X, y = make_classification(n_samples=100
                            ,n_features=2
                            ,n_redundant=0 #添加冗余特征
                            ,n_informative=2#包含信息的特征数
                            ,random_state=1#随机模式
                            ,n_clusters_per_class=1)#每个簇内包含的标签类别
#X是100行带有2个特征标签的数据，y是二分类标签
#也可以画一下散点图看一下
#plt.scatter(X[:, 0], X[:, 1])
#plt.show()
# 生成的二分类数据的两个簇距离很远，不利于我们测试分类器效果，因此我们使用np生成随机数据，使数据加减随机数，变得更加稀疏
rng = np.random.RandomState(2)# 生成一种随机模式
X += 2 * rng.uniform(size=X.shape)#加减0 1之间的随机数. uniform 默认生成01之间的随机数
linearly_separable = (X, y)
#plt.scatter(X[:, 0], X[:, 1])
# 用make_moons创建月亮型数据，make_circles创建环形数据，并将三组数据打包起来
datasets = [make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable]

In [ ]:
#  画出三种数据集和三颗决策树的分类效应图像
# 宽高比
figure = plt.figure(figsize=(6,9))
# 安排图像显示位置
i = 1
# 开始迭代数据，对datasets中的数据进行for循环
for ds_index, ds in enumerate(datasets):
    # 对x中的数据进行标准化处理，然后分训练集和测试集
    X, y = ds
    X = StandardScaler().fit_transform(X)
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4,
    random_state=42)

    # 找出特征的最大最小值然后放大一点范围
    x1_min, x1_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    x2_min, x2_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

    # 用特征向量生成网格数据，网格数据相当于坐标轴上无数个点
    # meshgrid 能够将两个一维数组生成二维数组
    # 第一个数组维度是n，第二个数组维度是m，生成的第一个二维数组是以ndarray为行，m行的矩阵，
    #第二个二维数组是以marray的转置为列，n列的矩阵
    # 生成的网格数据，用来绘制决策边界的，因为绘制决策边界的函数contourf要求输入的两个特征必须是二维数组
    array1, array2 = np.meshgrid(np.arange(x1_min, x1_max, 0.2),
                                    np.arange(x2_min, x2_max, 0.2))
    
    # 接下来生成彩色画布
    # 用listedcolormap为画布创建颜色
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])

    # 在画布上加一个子图，数据为len(datasets)行，2列，放在位置i上
    ax = plt.subplot(len(datasets), 2, i)
    # 到这里为止，已经生成01之间的坐标系3个了，接下来为坐标放上标题
    # 我们有三个坐标系，但我们只需要在第一个坐标系上有标题，因此设定if ds_index==0
    if ds_index == 0:
        ax.set_title('input data')

    # 将数据集的分布放到我们的坐标系上
    # 先放训练集
    ax.scatter(x_train[:, 0], x_train[:, 1], c=y_train,
            cmap=cm_bright, edgecolors='k')
    # 放测试集
    ax.scatter(x_valid[:, 0], x_valid[:, 1], c=y_valid,
                cmap=cm_bright, alpha=0.6, edgecolors='k')
    # 为图设置坐标轴的最大值和最小值，并设定没有坐标轴
    ax.set_xlim(array1.min(), array1.max())
    ax.set_ylim(array2.min(), array2.max())
    ax.set_xticks(())
    ax.set_yticks(())

    # 每次循环后，改变i的值让图每次位列不同的位置
    i += 1
    # 画出来三个散点图，第一行两个，第二行一个

########################迭代决策树模型###########################

#迭代决策树首先用subplot增加子图，subplot(行列索引)这样的结构，并使用索引i定义图的位置
#这里直接加i+1就是在2， 4， 6了
    ax = plt.subplot(len(datasets), 2, i)

    #决策树的建模过程
    clf = DecisionTreeClassifier(max_depth=5)
    clf = clf.fit(x_train, y_train)
    score = clf.score(x_valid, y_valid)
    # 绘制决策边界，为此我们将网格中的每个点指定一种颜色
    # 分类树的接口，predict_proba,返回每一个输入的数据点所对应的标签类概率
    # 剩下看笔记
    Z = clf.predict_proba(np.c_[array1.ravel(), array2.ravel()])[:, 1]
    # 将返回的类概率作为数据，放到contourf里面绘制轮廓
    Z = Z.reshape(array1.shape)
    ax.contourf(array1, array2, Z, cmap=cm, alpha=0.8)

    # 将数据集放到图中去
    ax.scatter(x_train[:, 0], x_train[:, 1], c=y_train, cmap=cm_bright, edgecolors='k')
    # 将测试集放到图中去
    ax.scatter(x_valid[:, 0], x_valid[:, 1], c=y_valid, cmap=cm_bright, edgecolors='k', alpha=0.6)

    # 位图设置
    ax.set_xlim(array1.min(), array1.max())
    ax.set_ylim(array2.min(), array2.max())
    
    ax.set_xticks(())
    ax.set_yticks(())
    
    if ds_index == 0:
        ax.set_title('DecisionTree')
    
    # 写在右下角的数字
    ax.text(array1.max() - 0.3, array2.min() + 0.3, ('{:.1f}%'.format(score*100)),
    size=15, horizontalalignment='right')

    i += 1
# 防止重叠
plt.tight_layout()
plt.show()


